In [10]:
import pandas as pd
from tqdm import tqdm
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

load_dotenv()

uri = os.getenv('uri')
user = os.getenv('user')
password = os.getenv('password')

driver= GraphDatabase.driver(uri, auth=(user, password))

### Loading ESCO datasets

In [3]:
occupations = pd.read_csv('ESCO_data/occupations.csv', dtype=str).fillna('').applymap(lambda x: x.lower())
isco_groups = pd.read_csv('ESCO_data/ISCOGroups.csv', dtype=str).fillna('').applymap(lambda x: x.lower())
occupations_hierarchy = pd.read_csv('ESCO_data/occupations_hierarchy.csv', dtype=str).fillna('').applymap(lambda x: x.lower())
skills_data = pd.read_csv('ESCO_data/skills.csv', dtype=str).fillna('').applymap(lambda x: x.lower())
skills_occupations_data = pd.read_csv('ESCO_data/occupation_skill_relations.csv',dtype=str).fillna('').applymap(lambda x:x.lower())



skills_occupations_data.sample(5)

,OCCUPATIONTYPE,OCCUPATIONID,RELATIONTYPE,SKILLID
72682,escooccupation,key_16926,optional,key_2210
74645,escooccupation,key_16973,essential,key_13999
83379,escooccupation,key_17185,essential,key_1781
61923,escooccupation,key_16675,optional,key_1800
16815,escooccupation,key_15577,optional,key_2624


## Ingesting data

In [8]:
from neo4j import GraphDatabase
from tqdm import tqdm

class Neo4jHandler:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def run_query(self, query):
        with self.driver.session() as session:
            result = session.run(query)
            return [record.data() for record in result]

    def create_occupation_nodes(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MERGE (c:Occupation {{key_id: '{row['ID']}'}})  
                ON CREATE SET 
                  c.occupation_name = "{row['PREFERREDLABEL']}",
                  c.origin_url = coalesce('{row['ORIGINURI']}', ""),
                  c.code = coalesce('{row['CODE']}', ""),
                  c.description = coalesce("{row['DESCRIPTION'].replace('"','')}", "")
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows

    def create_isco_group_nodes(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MERGE (c:ISCO_group {{key_id: '{row['ID']}'}})  
                ON CREATE SET 
                  c.occupation_name = "{row['PREFERREDLABEL']}",
                  c.origin_url = coalesce('{row['ORIGINURI']}', ""),
                  c.code = coalesce('{row['CODE']}', ""),
                  c.description = coalesce("{row['DESCRIPTION'].replace('"','')}", "")
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows

    def create_isco_hierarchy(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MATCH (a:ISCO_group {{key_id: '{row['PARENTID']}'}}), (b:ISCO_group {{key_id: '{row['CHILDID']}'}})
                MERGE (a)<-[r:HAS_PARENT_OCCUPATION]-(b)
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows

    def create_occupation_isco_hierarchy(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MATCH (a:ISCO_group {{key_id: '{row['PARENTID']}'}}), (b:Occupation {{key_id: '{row['CHILDID']}'}})
                MERGE (a)<-[r:HAS_PARENT_OCCUPATION]-(b)
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows

    def create_occupation_hierarchy(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MATCH (a:Occupation {{key_id: '{row['PARENTID']}'}}), (b:Occupation {{key_id: '{row['CHILDID']}'}})
                MERGE (a)<-[r:HAS_PARENT_OCCUPATION]-(b)
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows

    def create_skill_nodes(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MERGE (c:Skill {{key_id: '{row['ID']}'}})  
                ON CREATE SET 
                  c.skill_name = "{row['PREFERREDLABEL']}",
                  c.origin_url = coalesce('{row['ORIGINURI']}', ""),
                  c.skill_type = coalesce('{row['SKILLTYPE']}', ""),
                  c.description = coalesce("{row['DESCRIPTION'].replace('"','')}", "")
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows

    def create_skill_occupation_relation(self, df):
        failed_rows = []
        for _, row in tqdm(df.iterrows()):
            try:
                query = f"""
                MATCH (a:Occupation {{key_id: '{row['OCCUPATIONID']}'}}), (b:Skill {{key_id: '{row['SKILLID']}'}})
                MERGE (a)-[r:HAS_SKILL]->(b)
                SET r.type = '{row['RELATIONTYPE']}'
                """
                self.run_query(query)
            except Exception as e:
                print(f"Failed to process row: {row}, Error: {e}")
                failed_rows.append(row)
        return failed_rows


In [2]:
## ingesting Occupations
neo4j_handler = Neo4jHandler(uri, user, password)

failed_rows_occupation = neo4j_handler.create_occupation_nodes(occupations)
print('occupations ingested')

failed_rows_isco_group = neo4j_handler.create_isco_group_nodes(isco_groups) 
print('isco_groups ingested')

failed_rows_isco_group_hierarchy = neo4j_handler.create_isco_hierarchy(occupations_hierarchy) 
print('occupations_hierarchy ingested')
failed_rows_occupation_isco_hierarchy = neo4j_handler.create_occupation_isco_hierarchy(occupations_hierarchy)
print('occupations_hierarchy ingested')
failed_rows_occupation_hierarchy = neo4j_handler.create_occupation_hierarchy(occupations_hierarchy)
print('occupations_hierarchy ingested')

failed_row_skill = neo4j_handler.create_skill_nodes(skills_data)
print(f'failed_row_skill {failed_row_skill}')
failed_rows_skill_occupation_relation = neo4j_handler.create_skill_occupation_relation(skills_occupations_data)
print(f'failed_rows_skill_occupation_relation {failed_rows_skill_occupation_relation}')
neo4j_handler.close()

## Vector index creation on skill 

In [3]:
from langchain.vectorstores.neo4j_vector import Neo4jVector

## If OPENAI key is present
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = "text-embedding-3-small" 

## If Google API key is present 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [7]:
graph_vector_index = Neo4jVector.from_existing_graph(
    embedding = embeddings,
    url=uri,
    username=user,
    password=password,
    index_name='skills',
    node_label="Skill",
    text_node_properties=['skill_name','description'],
    embedding_node_property='embedding',
)

In [59]:
hr_topics = [
    "Employee relations management",
    "Recruitment and selection processes",
    "Performance management systems",
    "Compensation and benefits administration",
    "Labor law and employment regulations",
    "Training and development programs",
    "Conflict resolution and negotiation skills"
]


sales_skills = [
    "Customer relationship management (CRM)",
    "Sales strategies and techniques",
    "Negotiation and persuasion",
    "Product knowledge and presentation",
    "Market and competitor analysis",
    "Closing sales and follow-ups",
    "Communication and interpersonal skills"
]

topics = [
    "Financial analysis",
    "Investment management",
    "Portfolio management",
    "Risk management",
    "Asset allocation",
    "Equity research",
    "Fixed income analysis",
    "Quantitative analysis",
    "Performance measurement",
    "Investment strategy development",
    "Market analysis",
    "Financial modeling",
    "Valuation techniques",
    "Investment selection",
    "Asset pricing"
]



In [62]:
total_skills = []
for skill in topics:
    skill_lst = []
    result = vector_index.similarity_search(skill, k=3)
    skill_lst = [val.page_content.split('\n')[1].split(':')[-1].strip(' ') for val in result]
    total_skills.extend(skill_lst)

In [63]:
total_skills = list(set(total_skills))
total_skills


['economics',
 'follow a brief',
 'job market offers',
 'risk management',
 'manage financial risk',
 'manage several projects',
 'implement hedging strategies for clients',
 'parts pricing',
 'monitor stock market',
 'make investment decisions',
 'monitor bond market',
 'market analysis',
 'manage commercial risks',
 'perform market research',
 'market participants',
 'meet productivity targets',
 'games rules',
 'manage securities trading',
 'cloud monitoring and reporting',
 'operate financial instruments',
 'assist in fund management',
 'manage profitability',
 'advise on investment',
 'review investment portfolios',
 'perform financial analysis on price strategies',
 'analyse experimental laboratory data']

In [49]:
neo4j_handler = Neo4jHandler(uri, user, password)
occupation_dict = {}
non_matched_skills = []
for skill in total_skills:
    
    try:
        query = f'''MATCH (s:Skill {{skill_name:'{skill}'}})<-[:HAS_SKILL]-(o:Occupation) RETURN DISTINCT(o.occupation_name)'''
        result = neo4j_handler.run_query(query)
        occupation_lst = []
        occupation_lst = [val['(o.occupation_name)'] for val in result]
        
        for occupation in occupation_lst:
            if occupation in occupation_dict.keys():
                occupation_dict[occupation] += 1
            elif occupation not in occupation_dict.keys():
                occupation_dict[occupation] = 1
    except:
        non_matched_skills.append(skill)


In [50]:
top_3_jobs = sorted(occupation_dict.items(), key=lambda x: x[1], reverse=True)[:5]

# Print the top 3 keys and their values
for job, count in top_3_jobs:
    print(f"{job}: {count}")

human resources manager: 7
service manager: 6
corporate training manager: 5
marketing manager: 5
special-interest groups' official: 4


In [51]:
query = f'''MATCH (o:Occupation{{occupation_name:"human resources manager"}}) RETURN DISTINCT(o.description)'''
result = neo4j_handler.run_query(query)

In [52]:
result

[{'(o.description)': "human resources managers plan, design and implement processes related to the human capital of companies. they develop programs for recruiting, interviewing, and selecting employees based on a previous assessment of the profile and skills required in the company. moreover, they manage compensation and development programs for the company's employees comprising trainings, skill assessment and yearly evaluations, promotion, expat programs, and general assurance of the well-being of the employees in the workplace."}]